In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
import torch
from dnn_training.dataset_ak import *
from dnn_training.dataset_torch import *
import numpy as np
from dnn_training.trainer import *
import awkward as ak

In [2]:
device = "cuda:1"

In [3]:
model, loss = makeModelLoss()
trainer = Trainer(model.to(device), loss, device=device, log_output="/grid_mnt/data_cms_upgrade/cuisset/supercls/dnn_training")

In [4]:
train_dataset, val_dataset = makeDatasetsTrainVal("/grid_mnt/data_cms_upgrade/cuisset/supercls/alessandro_electrons/supercls-v13/superclsDumper_1.root", device="cpu")

In [10]:
trainer.full_train(train_dataset, val_dataset, nepochs=5)

  0%|          | 0/5 [00:00<?, ?it/s]

train_loss = 0.30187917076546
val_loss = 0.010887423530220985
train_loss = 0.30434493648499483
val_loss = 0.011025166371837258
train_loss = 0.28661539388122037
val_loss = 0.010727119981311262
train_loss = 0.3006898802414071
val_loss = 0.013658439042046666
train_loss = 0.2873406008075108
val_loss = 0.0105202526319772


In [6]:
trainer.scheduler.get_last_lr()

[0.001]

In [11]:
import torch.onnx

In [27]:
train_dataset[0:2]["features"][0].dtype

torch.float32

In [28]:
torch.ones((1, 17)).dtype

torch.float32

In [22]:
model.to("cpu")

torch.onnx.export(
    model,
    train_dataset[0:2]["features"][0].to("cpu"),
    "model.onnx",
    export_params=True,
    verbose=True,
    input_names=["input"],
    output_names=["output"],
    opset_version=10,
    dynamic_axes={"input" : {0:"batch_size"}, "output": {0:"batch_size"}}
)

Exported graph: graph(%input : Float(*, 17, strides=[17, 1], requires_grad=0, device=cpu),
      %0.weight : Float(100, 17, strides=[17, 1], requires_grad=1, device=cpu),
      %0.bias : Float(100, strides=[1], requires_grad=1, device=cpu),
      %2.weight : Float(200, 100, strides=[100, 1], requires_grad=1, device=cpu),
      %2.bias : Float(200, strides=[1], requires_grad=1, device=cpu),
      %4.weight : Float(100, 200, strides=[200, 1], requires_grad=1, device=cpu),
      %4.bias : Float(100, strides=[1], requires_grad=1, device=cpu),
      %6.weight : Float(1, 100, strides=[100, 1], requires_grad=1, device=cpu),
      %6.bias : Float(1, strides=[1], requires_grad=1, device=cpu)):
  %/0/Gemm_output_0 : Float(*, 100, strides=[100, 1], requires_grad=1, device=cpu) = onnx::Gemm[alpha=1., beta=1., transB=1, onnx_name="/0/Gemm"](%input, %0.weight, %0.bias), scope: torch.nn.modules.container.Sequential::/torch.nn.modules.linear.Linear::0 # /grid_mnt/data_cms_upgrade/cuisset/conda/envs/ti